In [0]:
SELECT
    al.borough,
    al.neighbourhood_name AS neighbourhood,
    COUNT(DISTINCT c.complaint_id) AS complaint_count,
    COUNT(DISTINCT f.listing_id)   AS airbnb_count,
    ROUND(
      COUNT(DISTINCT f.listing_id)::DECIMAL
      / NULLIF(COUNT(DISTINCT c.complaint_id), 0), 4
    ) AS airbnb_per_complaint
FROM public.dim_airbnb_location     al
LEFT JOIN public.fact_airbnb_listings f
  ON f.geohash = al.geohash
LEFT JOIN public.dim_311_location l311
  ON l311.geohash = al.geohash
LEFT JOIN public.fact_311_complaint c
  ON c.location_key = l311.location_key
WHERE al.borough IS NOT NULL
GROUP BY al.borough, al.neighbourhood_name
HAVING COUNT(DISTINCT c.complaint_id) > 10
ORDER BY airbnb_count DESC
LIMIT 50;

In [0]:
SELECT
    al.geohash,
    al.borough,
    COUNT(DISTINCT c.complaint_id) AS complaints,
    COUNT(DISTINCT f.listing_id)   AS airbnb_listings,
    ROUND(
      COUNT(DISTINCT c.complaint_id)::DECIMAL
      / NULLIF(COUNT(DISTINCT f.listing_id), 0), 2
    ) AS complaints_per_listing
FROM public.dim_airbnb_location al
LEFT JOIN public.fact_airbnb_listings f
  ON f.geohash = al.geohash
LEFT JOIN public.dim_311_location l311
  ON l311.geohash = al.geohash
LEFT JOIN public.fact_311_complaint c
  ON c.location_key = l311.location_key
GROUP BY al.geohash, al.borough
HAVING COUNT(DISTINCT f.listing_id) > 5
ORDER BY complaints_per_listing DESC
LIMIT 30;

In [0]:
SELECT
    d.quarter,
    d.month_name,
    l311.borough,
    COUNT(*) AS complaints,
    AVG(COUNT(*)) OVER (PARTITION BY d.quarter) AS avg_quarterly_complaints
FROM public.fact_311_complaint f
JOIN public.dim_date d
  ON f.created_date_key = d.date_id
JOIN public.dim_311_location l311
  ON f.location_key = l311.location_key
WHERE d.year >= 2023
GROUP BY d.quarter, d.month_name, l311.borough
ORDER BY d.quarter, l311.borough;

In [0]:
SELECT
    CASE WHEN d.is_weekend THEN 'Weekend' ELSE 'Weekday' END AS day_type,
    l311.borough,
    COUNT(*) AS complaint_count,
    ROUND(AVG(f.duration_hours), 2) AS avg_resolution_hours
FROM public.fact_311_complaint f
JOIN public.dim_date d
  ON f.created_date_key = d.date_id
JOIN public.dim_311_location l311
  ON f.location_key = l311.location_key
GROUP BY day_type, l311.borough
ORDER BY l311.borough, day_type;